<a href="https://colab.research.google.com/github/nicholasketo/Stock-Market-Value-Prediction/blob/main/Quant_momentum_investing_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import requests as rq
import math
from scipy import stats
import xlsxwriter


In [3]:
url = 'https://raw.githubusercontent.com/nicholasketo/Stock-Market-Value-Prediction/main/sp_500_stocks.csv'
stocks = pd.read_csv(url)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
IEX_CLOUD_API_TOKEN = 'sk_30a7d5719bf14aa584faa9c117f2988f'

In [5]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
try:
    response = rq.get(api_url)
    response.raise_for_status()  # Check for any HTTP errors
    data = response.json()  # Try to parse the response as JSON

    # If you want to print the data
    print(data)

except rq.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except ValueError as e:
    print(f"ValueError: {e}")

{'companyName': 'Apple Inc', 'marketcap': 2827801368640, 'week52high': 197.7, 'week52low': 123.48, 'week52highSplitAdjustOnly': 198.23, 'week52lowSplitAdjustOnly': 124.17, 'week52change': 0.31235149250930383, 'sharesOutstanding': 15552752000, 'float': 0, 'avg10Volume': 63492450, 'avg30Volume': 58016722, 'day200MovingAvg': 179.3, 'day50MovingAvg': 174.76, 'employees': 164000, 'ttmEPS': 6.13, 'ttmDividendRate': 0.7084164673375632, 'dividendYield': 0.00389625160784052, 'nextDividendDate': '2023-11-10', 'exDividendDate': '2023-08-11', 'nextEarningsDate': '2024-02-01', 'peRatio': 29.154094217640083, 'beta': 0, 'maxChangePercent': 69.87946358958365, 'year5ChangePercent': 2.626391160397304, 'year2ChangePercent': 0.2171838347918296, 'year1ChangePercent': 0.3178313115038076, 'ytdChangePercent': 0.4072363308762295, 'month6ChangePercent': 0.05181499116931243, 'month3ChangePercent': 0.01935324721923215, 'month1ChangePercent': 0.02439574060510452, 'day30ChangePercent': 0.02439574060510452, 'day5Cha

In [6]:
data['year1ChangePercent']

0.3178313115038076

In [7]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [8]:

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = rq.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol in data and 'quote' in data[symbol] and 'stats' in data[symbol]:
            final_dataframe = final_dataframe.append(
                pd.Series([symbol,
                           data[symbol]['quote'].get('latestPrice', 'N/A'),
                           data[symbol]['stats'].get('year1ChangePercent', 'N/A'),
                           'N/A'],
                          index=my_columns),
                ignore_index=True
            )
        else:
            print(f"Data not found for symbol: {symbol}")

final_dataframe

<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

Data not found for symbol: ABC
Data not found for symbol: ANTM


<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

Data not found for symbol: DISCA
Data not found for symbol: FBHS


<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

...

Data not found for symbol: RE


<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

Data not found for symbol: VIAC
Data not found for symbol: WLTW


<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,109.39,-0.193502,N/A
1,AAL,12.01,-0.179469,N/A
2,AAP,57.64,-0.670976,N/A
3,AAPL,182.89,0.317831,N/A
4,ABBV,142.03,-0.001331,N/A
...,...,...,...,...
491,YUM,125.61,0.043362,N/A
492,ZBH,104.86,-0.007333,N/A
493,ZBRA,206.44,-0.119817,N/A
494,ZION,33.76,-0.285414,N/A


In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [12]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/ final_dataframe.loc[i, 'Price'])

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,109.39,-0.193502,18
1,AAL,12.01,-0.179469,167
2,AAP,57.64,-0.670976,34
3,AAPL,182.89,0.317831,11
4,ABBV,142.03,-0.001331,14
...,...,...,...,...
491,YUM,125.61,0.043362,16
492,ZBH,104.86,-0.007333,19
493,ZBRA,206.44,-0.119817,9
494,ZION,33.76,-0.285414,59


In [14]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'

]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = rq.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data and 'quote' in data[symbol] and 'stats' in data[symbol]:
            hqm_dataframe = hqm_dataframe.append(
                pd.Series([
                   symbol,
                   data[symbol]['quote'].get('latestPrice', 'N/A'),
                  'N/A',
                   data[symbol]['stats'].get('year1ChangePercent', 'N/A'),
                  'N/A',
                  data[symbol]['stats'].get('month6ChangePercent', 'N/A'),
                  'N/A',
                  data[symbol]['stats'].get('month3ChangePercent', 'N/A'),
                  'N/A',
                  data[symbol]['stats'].get('month1ChangePercent', 'N/A'),
                  'N/A',
                  'N/A'
                ],
                index=hqm_columns),
                ignore_index=True
            )
        else:
            print(f"Data not found for symbol: {symbol}")

hqm_dataframe



<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

Data not found for symbol: ABC
Data not found for symbol: ANTM


<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

Data not found for symbol: DISCA


<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

...

Data not found for symbol: RE


<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

Data not found for symbol: VIAC
Data not found for symbol: WLTW


<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-14-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,109.39,N/A,-0.193502,N/A,-0.167484,N/A,-0.124473,N/A,-0.000904,N/A,N/A
1,AAL,12.01,N/A,-0.179469,N/A,-0.152848,N/A,-0.265166,N/A,-0.079154,N/A,N/A
2,AAP,57.64,N/A,-0.670976,N/A,-0.524326,N/A,-0.177194,N/A,0.128412,N/A,N/A
3,AAPL,182.89,N/A,0.317831,N/A,0.051815,N/A,0.019353,N/A,0.024396,N/A,N/A
4,ABBV,142.03,N/A,-0.001331,N/A,-0.019289,N/A,-0.044882,N/A,-0.031416,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,125.61,N/A,0.043362,N/A,-0.078798,N/A,-0.066924,N/A,0.050393,N/A,N/A
492,ZBH,104.86,N/A,-0.007333,N/A,-0.228942,N/A,-0.154979,N/A,-0.044901,N/A,N/A
493,ZBRA,206.44,N/A,-0.119817,N/A,-0.240175,N/A,-0.161906,N/A,-0.070628,N/A,N/A
494,ZION,33.76,N/A,-0.285414,N/A,0.47734,N/A,-0.095546,N/A,0.012171,N/A,N/A


In [15]:
%pip install stats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.6/127.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stats: filename=stats-0.1.2a0-py3-none-any.whl size=24283 sha256=45b051be89b466f2a84cfe7a04f275ebe32d919fd9e72dd492f00fb680e3951c
  Stored in directory: /root/.cache/pip/wheels/c9/85/bc/3371b9bce1e4f7b8b638e8c968a4dbd74db171ee180c48f808
Successfully built stats


In [26]:
import numpy as np
from scipy import stats

time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        data = hqm_dataframe[change_col].dropna()  # Drop rows with missing values

        if data.empty:
            print(f"No data available for {change_col}")
        else:
            score = hqm_dataframe.loc[row, change_col]
            if score is not None:
                hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(data, score) / 100
            else:
                print(f"No valid data available for {change_col}")

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

# Print the entire DataFrame
hqm_dataframe


0      1.0
1     0.94
2     0.88
3     0.92
4     0.98
5     0.84
6     0.96
7     0.82
8     0.52
9     0.76
10    0.66
11     0.9
12    0.74
13    0.86
14    0.48
15    0.78
16     0.3
17    0.56
18    0.68
19     0.7
20    0.54
21    0.28
22    0.44
23    0.44
24    0.44
25    0.38
26    0.62
27    0.14
28     0.5
29     0.6
30     0.8
31     0.4
32    0.24
33    0.72
34    0.58
35     0.2
36    0.34
37    0.32
38    0.36
39    0.04
40    0.02
41    0.12
42    0.16
43    0.26
44    0.06
45    0.22
46    0.64
47    0.18
48    0.08
49     0.1
Name: One-Year Return Percentile, dtype: object
0      1.0
1     0.98
2     0.86
3     0.92
4     0.96
5     0.94
6      0.9
7     0.72
8      0.7
9     0.84
10     0.8
11    0.88
12    0.82
13    0.62
14    0.76
15    0.56
16    0.42
17    0.74
18    0.68
19    0.48
20    0.64
21    0.28
22    0.52
23    0.52
24    0.52
25    0.26
26    0.38
27    0.78
28     0.6
29    0.58
30    0.32
31    0.46
32    0.22
33    0.34
34     0.4
35     0.3
36    

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,8,ADBE,585.31,N/A,0.953662,1.0,0.679679,1.0,0.104714,0.92,0.111111,0.9,0.96748
1,36,ANET,209.60,N/A,0.628121,0.94,0.540803,0.98,0.185921,1.0,0.095763,0.88,0.956809
2,19,AIZ,159.53,N/A,0.347195,0.88,0.228559,0.86,0.12574,0.94,0.093651,0.86,0.906504
3,35,AMZN,142.08,N/A,0.576384,0.92,0.350717,0.92,0.003445,0.72,0.11527,0.92,0.903455
4,48,AVGO,911.13,N/A,0.949811,0.98,0.447907,0.96,0.006298,0.74,0.062182,0.8,0.881606
5,21,AKAM,110.56,N/A,0.298116,0.84,0.385692,0.94,0.141928,0.96,0.029312,0.66,0.851118
6,30,AMD,113.59,N/A,0.79851,0.96,0.262801,0.9,-0.028765,0.6,0.057907,0.78,0.832825
7,20,AJG,245.57,N/A,0.284485,0.82,0.147021,0.72,0.091595,0.9,0.051679,0.72,0.82876
8,25,ALL,129.28,N/A,0.037007,0.52,0.139151,0.7,0.180712,0.98,0.145913,0.96,0.819106
9,16,AFL,80.32,N/A,0.222807,0.76,0.212609,0.84,0.044764,0.86,0.043557,0.7,0.809451


In [27]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True)
hqm_dataframe

<ipython-input-27-ccf72467b85c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)


,level_0,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,0,8,ADBE,585.31,N/A,0.953662,1.0,0.679679,1.0,0.104714,0.92,0.111111,0.9,0.96748
1,1,36,ANET,209.60,N/A,0.628121,0.94,0.540803,0.98,0.185921,1.0,0.095763,0.88,0.956809
2,2,19,AIZ,159.53,N/A,0.347195,0.88,0.228559,0.86,0.12574,0.94,0.093651,0.86,0.906504
3,3,35,AMZN,142.08,N/A,0.576384,0.92,0.350717,0.92,0.003445,0.72,0.11527,0.92,0.903455
4,4,48,AVGO,911.13,N/A,0.949811,0.98,0.447907,0.96,0.006298,0.74,0.062182,0.8,0.881606
5,5,21,AKAM,110.56,N/A,0.298116,0.84,0.385692,0.94,0.141928,0.96,0.029312,0.66,0.851118
6,6,30,AMD,113.59,N/A,0.79851,0.96,0.262801,0.9,-0.028765,0.6,0.057907,0.78,0.832825
7,7,20,AJG,245.57,N/A,0.284485,0.82,0.147021,0.72,0.091595,0.9,0.051679,0.72,0.82876
8,8,25,ALL,129.28,N/A,0.037007,0.52,0.139151,0.7,0.180712,0.98,0.145913,0.96,0.819106
9,9,16,AFL,80.32,N/A,0.222807,0.76,0.212609,0.84,0.044764,0.86,0.043557,0.7,0.809451


In [25]:
import pandas as pd

# Assuming hqm_dataframe and time_periods are defined

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(pd.to_numeric(hqm_dataframe.loc[row, f'{time_period} Return Percentile'], errors='coerce'))
    momentum_percentiles = [0 if pd.isnull(x) else x for x in momentum_percentiles]
    hqm_dataframe.loc[row, 'HQM Score'] = sum(momentum_percentiles) / len(momentum_percentiles)

hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,8,ADBE,585.31,N/A,0.953662,0.993902,0.679679,0.997967,0.104714,0.949187,0.111111,0.928862,0.96748
1,36,ANET,209.60,N/A,0.628121,0.965447,0.540803,0.99187,0.185921,0.977642,0.095763,0.892276,0.956809
2,19,AIZ,159.53,N/A,0.347195,0.886179,0.228559,0.892276,0.12574,0.95935,0.093651,0.888211,0.906504
3,35,AMZN,142.08,N/A,0.576384,0.961382,0.350717,0.96748,0.003445,0.752033,0.11527,0.932927,0.903455
4,48,AVGO,911.13,N/A,0.949811,0.99187,0.447907,0.979675,0.006298,0.760163,0.062182,0.794715,0.881606
5,21,AKAM,110.56,N/A,0.298116,0.847561,0.385692,0.973577,0.141928,0.96748,0.029312,0.615854,0.851118
6,30,AMD,113.59,N/A,0.79851,0.989837,0.262801,0.928862,-0.028765,0.642276,0.057907,0.770325,0.832825
7,20,AJG,245.57,N/A,0.284485,0.837398,0.147021,0.796748,0.091595,0.934959,0.051679,0.745935,0.82876
8,25,ALL,129.28,N/A,0.037007,0.542683,0.139151,0.788618,0.180712,0.97561,0.145913,0.969512,0.819106
9,16,AFL,80.32,N/A,0.222807,0.778455,0.212609,0.880081,0.044764,0.869919,0.043557,0.70935,0.809451


In [17]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        data = hqm_dataframe[change_col].dropna()  # Remove rows with missing values
        if data.empty:
            print(f"No data available for {change_col}")
        else:
            filtered_data = data[data.notnull()]
            hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(filtered_data, hqm_dataframe.loc[row, change_col]) / 100

hqm_dataframe

TypeError: ignored

In [ ]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']
for row in hqm_dataframe.index:
  for time_period in time_periods:
    change_col = f'{time_period} Price Return'
    percentile_col = f'{time_period} Return Percentile'
    hqm_dataframe.loc[row,percentile_col] = stats.percentileofscore(hqm_dataframe[change_col],hqm_dataframe.loc[row, change_col])/100

hqm_dataframe